Following is the Assignment 2 task completed by 
1. Suraj Sharma (150914923)
2. Albin lamichhane (151229947)

In [ ]:
 !nvidia-smi

Wed Nov 23 08:27:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Importing the required library for the task
import pandas as pd
import numpy as np
import scipy.stats
from tqdm import tqdm
import itertools
from IPython.display import display

tqdm.pandas()





In [ ]:
#mounting the the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# naming of required column for all data
columns_name = {'data' : ["user_id", "item_id", "rating", "timestamp"] ,
                'item' : ['movie_id', 'movie_title' , 'release_date' , 'video_release_date' ,
              'IMDb_URL' , 'unknown' , 'Action', 'Adventure' , 'Animation' ,
              'Children', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film_Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ],
              'user': ['user_id' , 'age' , 'gender' , 'occupation' , 'zip code'],
              'genre' : ['genre_name', 'genre_ID'] 
              }


In [ ]:
#reading data from the file of google drive
data = pd.read_csv('/content/drive/MyDrive/recommenderassignment1/u.data', "\t", encoding="utf-8", names= columns_name['data'])
print(data.head(5))
print("---------------------------------------------")
print("Number of row of the given data is: ", len(data.user_id))
print("---------------------------------------------")
print(data.info())
print("---------------------------------------------")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
---------------------------------------------
Number of row of the given data is:  100000
---------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   item_id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB
None
---------------------------------------------


In [ ]:
# reading movies file from the drive
movies = pd.read_csv("/content/drive/MyDrive/recommenderassignment1/u.item", sep = '|' ,encoding="latin-1", names= columns_name['item'])
movies = movies[0:100]
print('----------------------------------------------------')
print(movies.head(2))
print("----------------------------------------------------")
print("Total number of given movies is : ",len(movies))
print("----------------------------------------------------")
movies.drop(['video_release_date', 'release_date', 'IMDb_URL'], axis=1, inplace = True)
print(movies.head(2))
print("----------------------------------------------------")

----------------------------------------------------
   movie_id       movie_title release_date  video_release_date  \
0         1  Toy Story (1995)  01-Jan-1995                 NaN   
1         2  GoldenEye (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   

   Adventure  Animation  Children  ...  Fantasy  Film_Noir  Horror  Musical  \
0          0          1         1  ...        0          0       0        0   
1          1          0         0  ...        0          0       0        0   

   Mystery  Romance  Sci-Fi  Thriller  War  Western  
0        0        0       0         0    0        0  
1        0        0       0         1    0        0  

[2 rows x 24 columns]
----------------------------------------------------
Total number of given movies is :  100
-------------

In [ ]:
#reading user file from the drive
user = pd.read_csv('/content/drive/MyDrive/recommenderassignment1/u.user', sep = "|", encoding="utf-8", names= columns_name['user'])
print(user.head(2))
print("----------------------------------------------------")
print(len(user.user_id))
print("----------------------------------------------------")
genre = pd.read_csv('/content/drive/MyDrive/recommenderassignment1/u.genre', "|", encoding="utf-8", names= columns_name['genre'])
print(genre.head(2))


   user_id  age gender  occupation zip code
0        1   24      M  technician    85711
1        2   53      F       other    94043
----------------------------------------------------
943
----------------------------------------------------


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


  genre_name  genre_ID
0    unknown         0
1     Action         1


In [ ]:
def correlation_pearson(usera,userb):
  #step 1get all ratings by 2 users
  usera_ratings = data[data['user_id'] == usera].sort_values(by=['item_id'])
  userb_ratings = data[data['user_id'] == userb].sort_values(by=['item_id'])
  #step 2 get movie id watched by both users dubai le hereko movie ko dataframe
  userab = usera_ratings[usera_ratings["item_id"].isin(userb_ratings["item_id"])].sort_values(by=['item_id'])
  userba = userb_ratings[userb_ratings["item_id"].isin(usera_ratings["item_id"])].sort_values(by=['item_id'])
  #mean
  usera_mean = np.mean(usera_ratings['rating'])
  userb_mean = np.mean(userb_ratings['rating'])
  #Subtract
  var_a = np.array(np.subtract(userab['rating'],[usera_mean]))
  var_b = np.array(np.subtract(userba['rating'],[userb_mean]))
  #correlation formula
  first =np.sum(var_a*var_b)
  second =np.sqrt(np.sum(np.power(var_a,2)))*np.sqrt(np.sum(np.power(var_b,2)))
  correlation = first/second
  return correlation,usera,userb

  



Calculate the prediction function
𝒑𝒓𝒆𝒅(𝒂, 𝒑) = 𝒓¯
𝒂 +
∑𝒃 ∈𝑵 𝒔𝒊𝒎(𝒂, 𝒃) ∗ (𝒓𝒃,𝒑 − 𝒓¯
𝒃)/
∑𝒃 ∈𝑵 𝒔𝒊𝒎(𝒂, 𝒃)


In [ ]:
#Calculate the item score for a user based on another user
def item_score_userab(usera,userb,item_id):
  #get user rating for an item
  usera_ratings = data[data['user_id'] == usera].sort_values(by=['item_id'])
  userb_ratings = data[data['user_id'] == userb].sort_values(by=['item_id'])
  #step 2 get movie id watched by both users dubai le hereko movie ko dataframe
  userba = userb_ratings[userb_ratings["item_id"].isin(usera_ratings["item_id"])].sort_values(by=['item_id'])
  #mean of user b
  userb_mean = np.mean(usera_ratings['rating'])
  #pearson correlation 
  correlation, usera ,userb =correlation_pearson(usera,userb)
  var_b = float(userb_ratings[userb_ratings["item_id"] == item_id]["rating"]) - userb_mean
  #output
  first = correlation *var_b
  second = correlation
  return [first,second]






In [ ]:


def user_movie_predict(user_id,item_id):
  #user ratings
   user_ratings = data[data['user_id'] == user_id].sort_values(by=['item_id'])
   #get mean user ratings
   user_ratings_mean= np.mean(user_ratings['rating'])
   #get other user who have rated the movie
   other_users = data[data['item_id'] ==item_id]
   #calculate item score
   score = other_users.apply(lambda user:item_score_userab(user_id,user['user_id'],item_id),axis=1, result_type='expand' )
   score= np.array(score)
   score = score[~np.isnan(score).any(axis=1), :]
   prediction_score = user_ratings_mean + np.sum(score[:,0]) / np.sum(score[:,1])

   return item_id, movies.at[item_id - 1, 'movie_title'], prediction_score




In [ ]:
def get_user_prediction(user_id):

  movies_score=movies.progress_apply(lambda movie:user_movie_predict(user_id,movie['movie_id']),axis=1,result_type="expand")
  movies_score.columns = ['movie_id',"movie_title","prediction_rating"]
  return movies_score




In [ ]:
def get_movie_genres(movie_id):
    movie = movies.loc[(movies['movie_id']==movie_id)]
    genres = movie.values[0][3:]
    genres_index = np.argwhere(genres==1)
    return genres_index

In [ ]:
def get_movies_groupby_genre():
    genres = {k: [] for k in range(19)}
    for index, row in movies.iterrows():
        row = np.array(row[3:])
        genres_index = np.argwhere(row==1)
        for i in genres_index:
            genres[i[0]].append(index+1)
    return genres 

# **Task 2: Both Aggregration method**

In [ ]:
def both_aggregation(users, users_ratings):
    # create dataframe with all 3 users' predicted ratings
    data = [users_ratings[0]["movie_id"], users_ratings[0]["movie_title"], users_ratings[0]["prediction_rating"], users_ratings[1]["prediction_rating"], users_ratings[2]["prediction_rating"]]
    headers = ["movie id", "movie title", "user{} rating".format(str(users[0])), "user{} rating".format(str(users[1])), "user{} rating".format(str(users[2]))]
    all_users_ratings = pd.concat(data, axis=1, keys=headers)

    # remove ratings under threshold
    all_users_ratings = all_users_ratings[all_users_ratings[f"user{str(users[0])} rating"] >= 2]
    all_users_ratings = all_users_ratings[all_users_ratings[f"user{str(users[1])} rating"] >= 2]
    all_users_ratings = all_users_ratings[all_users_ratings[f"user{str(users[2])} rating"] >= 2]

    all_users_ratings['average'] = all_users_ratings.iloc[:, 1:4].mean(axis=1)
    all_users_ratings['minimum'] = all_users_ratings.iloc[:, 1:4].min(axis=1)
    # 1st iteration of sequential recommendation: score = average rating
    all_users_ratings['score'] = all_users_ratings.iloc[:, 1:4].mean(axis=1)

    return all_users_ratings

In [ ]:
def get_user_ratings_by_genre(user_ratings, movies_num):
    movies_grouby_genre = get_movies_groupby_genre()
    user_ratings = user_ratings.sort_values(by=['prediction_rating'], ascending=False).head(movies_num)
    score_grouby_genre = {k: 0 for k in range(19)}
    for genre_key in movies_grouby_genre.keys():
        for movie in movies_grouby_genre[genre_key]:
            score = user_ratings.loc[user_ratings["movie_id"]==movie]['prediction_rating']
            if not score.empty:
                score_grouby_genre[genre_key] += int(score)
    return score_grouby_genre        

In [ ]:
def get_group_ratings_by_genre(group_ratings, movies_num):
    movies_grouby_genre = get_movies_groupby_genre()
    group_ratings = group_ratings.sort_values(by=['score'], ascending=False).head(movies_num)
    score_grouby_genre = {k: 0 for k in range(19)}
    for genre_key in movies_grouby_genre.keys():
        for movie in movies_grouby_genre[genre_key]:
            score = group_ratings.loc[group_ratings["movie id"]==movie]['score']
            if not score.empty:
                score_grouby_genre[genre_key] += int(score)
    return score_grouby_genre 

In [ ]:
def user_satisfaction(user, user_ratings, group_ratings, movies_num):
    top_user_ratings = user_ratings.sort_values(by=['prediction_rating'], ascending=False).head(movies_num)

    user_filtered_group_ratings = group_ratings.sort_values(by=['score'], ascending=False).head(movies_num)
    group_list_satisfaction = user_filtered_group_ratings["user{} rating".format(str(user))].sum()

    user_list_satisfaction = top_user_ratings["prediction_rating"].sum()
    
    return group_list_satisfaction/user_list_satisfaction

In [ ]:
def group_satisfaction(users_list, users_ratings, group_ratings, movies_num):
    user_satisfactions = []
    for (index, user) in enumerate(users_list):
        user_satisfactions.append(user_satisfaction(user, users_ratings[index], group_ratings, movies_num))
    
    # return average satisfaction and weight (alpha)
    return [sum(user_satisfactions)/len(user_satisfactions), max(user_satisfactions) - min(user_satisfactions)]

In [ ]:
def group_genre_satisfaction(group_ratings_by_genre, users_ratings_by_genre):
    users_satisfactions_by_genre = []
    for user_ratings_by_genre in users_ratings_by_genre:
        satisfactions = {k: 0 for k in range(19)}
        for key in satisfactions.keys():
            if user_ratings_by_genre[key]:
                satisfactions[key] = group_ratings_by_genre[key]/user_ratings_by_genre[key] 
            else:
                satisfactions[key] = 0
        users_satisfactions_by_genre.append(satisfactions)
        
    output = {k: 0 for k in range(19)}
    
    for i in range(19):
        users_satisfactions_genre_i = tuple(d[i] for d in users_satisfactions_by_genre)
        output[i] = max(users_satisfactions_genre_i) - min(users_satisfactions_genre_i)
    
    return output

In [ ]:
def get_genre_alpha(genre_satisfaction, group_ratings):
    movies_id = list(group_ratings["movie id"])
    genre_alpha = {k: 0 for k in movies_id}
    for movie_id in movies_id:
        genres = get_movie_genres(movie_id)
        for genre in genres:
            genre_alpha[movie_id] += genre_satisfaction[genre[0]]
            
    return pd.DataFrame.from_dict(genre_alpha, orient="index")

In [ ]:
def sequential_iteration(users_list, users_ratings, rated_movies, group_ratings, iteration, movies_num, group_ratings_by_genre, users_ratings_by_genre):    
    filtered_group_ratings = group_ratings[group_ratings.index.isin(rated_movies.index)]
    genre_satisfaction = group_genre_satisfaction(group_ratings_by_genre, users_ratings_by_genre)
    
    genre_alpha = get_genre_alpha(genre_satisfaction, group_ratings)
    alpha = 0
    if iteration > 1:
        alpha = group_satisfaction(users_list, users_ratings, group_ratings, movies_num)[1]  
    filtered_group_ratings["score"] = (1-genre_alpha[0])*(1-alpha)*filtered_group_ratings["average"] + genre_alpha[0]*alpha*filtered_group_ratings["minimum"]

    filtered_group_ratings = filtered_group_ratings.sort_values(by=['score'], ascending=False)
    return filtered_group_ratings

In [ ]:
def sequential_recommender(users_list, sequences_num, movies_num):
    users_ratings = []
    users_ratings_by_genre = []
    for user in users_list:
        user_ratings =get_user_prediction(user)
        user_ratings_by_genre = get_user_ratings_by_genre(user_ratings, movies_num)
        users_ratings.append(user_ratings)
        users_ratings_by_genre.append(user_ratings_by_genre)
        
    rated_movies = pd.concat(users_ratings, axis=0)
    rated_movies = rated_movies[~rated_movies.index.duplicated(keep='first')]
    
    group_ratings = both_aggregation(users_list, users_ratings)
    group_ratings_by_genre = get_group_ratings_by_genre(group_ratings, movies_num)
    recommended_sequence = []
    for i in range(1, sequences_num+1):
        group_ratings_iteration = sequential_iteration(users_list, users_ratings, rated_movies, group_ratings, i, movies_num, group_ratings_by_genre, users_ratings_by_genre)
        recommended_sequence.append(group_ratings_iteration.head(movies_num))
        group_ratings = group_ratings_iteration

    return recommended_sequence

In [ ]:
recommended_sequence = sequential_recommender([2, 17, 35], 5, 20)

100%|██████████| 100/100 [01:24<00:00,  1.18it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


In [ ]:
display(recommended_sequence[0])
display(recommended_sequence[1])
display(recommended_sequence[2])
display(recommended_sequence[3])
display(recommended_sequence[4])

movie id                          movie title  user2 rating  \
59        60            Three Colors: Blue (1993)      4.154768   
58        59             Three Colors: Red (1994)      4.133487   
6          7                Twelve Monkeys (1995)      3.824165   
60        61           Three Colors: White (1994)      3.827472   
86        87   Searching for Bobby Fischer (1993)      3.870970   
57        58                     Quiz Show (1994)      3.660584   
46        47                       Ed Wood (1994)      3.765932   
64        65   What's Eating Gilbert Grape (1993)      3.570036   
9         10                   Richard III (1995)      3.207726   
19        20            Angels and Insects (1995)      3.672453   
30        31                  Crimson Tide (1995)      3.592571   
87        88          Sleepless in Seattle (1993)      3.339092   
61        62                      Stargate (1994)      3.100282   
52        53          Natural Born Killers (1994)      2.996797   
84        85                      Ref, The (1994)      3.011596   
7          8                          Babe (1995)      3.947538   
89        90  So I Married an Axe Murderer (1993)      2.888422   
3          4                    Get Shorty (1995)      3.560748   
27        28                     Apollo 13 (1995)      3.908753   
97        98     Silence of the Lambs, The (1991)      4.318462   

    user17 rating  user35 rating   average   minimum     score  
59       4.317539       3.643469  4.236154  4.154768  3.538434  
58       4.116447       3.787682  4.124967  4.116447  3.445561  
6        4.365747       4.446750  4.094956  3.824165  3.420493  
60       4.135971       3.498965  3.981722  3.827472  3.325909  
86       3.985695       5.837227  3.928333  3.870970  3.281313  
57       3.905927       4.219749  3.783256  3.660584  3.160131  
46       3.739160       3.497279  3.752546  3.739160  3.134480  
64       3.603156       5.576614  3.586596  3.570036  2.995863  
9        3.940367       4.359294  3.574046  3.207726  2.985380  
19       3.454633       4.496331  3.563543  3.454633  2.976606  
30       3.484338       3.675575  3.538454  3.484338  2.955650  
87       3.276192       2.506384  3.307642  3.276192  2.762854  
61       3.148662       2.443296  3.124472  3.100282  2.609853  
52       3.186865       4.773060  3.091831  2.996797  2.582588  
84       2.902469       4.177314  2.957033  2.902469  1.478516  
7        4.081529       4.352294  4.014534  3.947538  1.346050  
89       3.442656       2.388779  3.165539  2.888422  0.949662  
3        3.675893       4.398645  3.618320  3.560748  0.673176  
27       3.846604       2.579074  3.877679  3.846604  0.508548  
97       4.341874       4.435086  4.330168  4.318462  0.249055

movie id                         movie title  user2 rating  user17 rating  \
59        60           Three Colors: Blue (1993)      4.154768       4.317539   
58        59            Three Colors: Red (1994)      4.133487       4.116447   
6          7               Twelve Monkeys (1995)      3.824165       4.365747   
60        61          Three Colors: White (1994)      3.827472       4.135971   
86        87  Searching for Bobby Fischer (1993)      3.870970       3.985695   
97        98    Silence of the Lambs, The (1991)      4.318462       4.341874   
57        58                    Quiz Show (1994)      3.660584       3.905927   
46        47                      Ed Wood (1994)      3.765932       3.739160   
7          8                         Babe (1995)      3.947538       4.081529   
44        45          Eat Drink Man Woman (1994)      4.164948       4.252097   
64        65  What's Eating Gilbert Grape (1993)      3.570036       3.603156   
19        20           Angels and Insects (1995)      3.672453       3.454633   
30        31                 Crimson Tide (1995)      3.592571       3.484338   
9         10                  Richard III (1995)      3.207726       3.940367   
13        14                  Postino, Il (1994)      4.052797       4.251970   
27        28                    Apollo 13 (1995)      3.908753       3.846604   
47        48                  Hoop Dreams (1994)      4.116555       4.095128   
11        12          Usual Suspects, The (1995)      4.446519       4.682449   
85        86      Remains of the Day, The (1993)      4.079423       3.862350   
87        88         Sleepless in Seattle (1993)      3.339092       3.276192   

    user35 rating   average   minimum     score  
59       3.643469  4.236154  4.154768  2.230341  
58       3.787682  4.124967  4.116447  2.177140  
6        4.446750  4.094956  3.824165  2.141497  
60       3.498965  3.981722  3.827472  2.090513  
86       5.837227  3.928333  3.870970  2.069639  
97       4.435086  4.330168  4.318462  2.000299  
57       4.219749  3.783256  3.660584  1.988116  
46       3.497279  3.752546  3.739160  1.980153  
7        4.352294  4.014534  3.947538  1.931735  
44       4.319629  4.208522  4.164948  1.916783  
64       5.576614  3.586596  3.570036  1.892299  
19       4.496331  3.563543  3.454633  1.873157  
30       3.675575  3.538454  3.484338  1.864048  
9        4.359294  3.574046  3.207726  1.859271  
13       4.618645  4.152384  4.052797  1.831348  
27       2.579074  3.877679  3.846604  1.807230  
47       4.152270  4.105841  4.095128  1.796539  
11       3.429083  4.564484  4.446519  1.754662  
85       4.169397  3.970886  3.862350  1.744798  
87       2.506384  3.307642  3.276192  1.743901

movie id                       movie title  user2 rating  user17 rating  \
49        50                  Star Wars (1977)      4.609644       4.626430   
99       100                      Fargo (1996)      4.596102       4.819830   
63        64  Shawshank Redemption, The (1994)      4.452792       4.512652   
11        12        Usual Suspects, The (1995)      4.446519       4.682449   
97        98  Silence of the Lambs, The (1991)      4.318462       4.341874   
21        22                 Braveheart (1995)      4.139410       4.124113   
22        23                Taxi Driver (1976)      4.212438       4.373822   
59        60         Three Colors: Blue (1993)      4.154768       4.317539   
44        45        Eat Drink Man Woman (1994)      4.164948       4.252097   
88        89               Blade Runner (1982)      4.135199       4.088676   
47        48                Hoop Dreams (1994)      4.116555       4.095128   
58        59          Three Colors: Red (1994)      4.133487       4.116447   
13        14                Postino, Il (1994)      4.052797       4.251970   
82        83     Much Ado About Nothing (1993)      4.108987       4.022820   
55        56               Pulp Fiction (1994)      4.093374       4.294289   
92        93   Welcome to the Dollhouse (1995)      3.946802       3.913879   
6          7             Twelve Monkeys (1995)      3.824165       4.365747   
8          9           Dead Man Walking (1995)      4.018546       4.183173   
7          8                       Babe (1995)      3.947538       4.081529   
60        61        Three Colors: White (1994)      3.827472       4.135971   

    user35 rating   average   minimum     score  
49       3.909395  4.618037  4.609644  2.363104  
99       4.710713  4.707966  4.596102  2.287435  
63       3.941884  4.482722  4.452792  2.282395  
11       3.429083  4.564484  4.446519  2.227220  
97       4.435086  4.330168  4.318462  2.175268  
21       3.309282  4.131761  4.124113  2.155071  
22       2.920528  4.293130  4.212438  2.117951  
59       3.643469  4.236154  4.154768  2.099647  
44       4.319629  4.208522  4.164948  2.099319  
88       5.220215  4.111938  4.088676  2.086882  
47       4.152270  4.105841  4.095128  2.070948  
58       3.787682  4.124967  4.116447  2.050410  
13       4.618645  4.152384  4.052797  2.041980  
82       4.087805  4.065903  4.022820  2.041350  
55       4.553403  4.193832  4.093374  2.032504  
92       3.701667  3.930340  3.913879  2.016385  
6        4.446750  4.094956  3.824165  2.013711  
8        4.653366  4.100859  4.018546  2.010992  
7        4.352294  4.014534  3.947538  1.990252  
60       3.498965  3.981722  3.827472  1.967082

movie id                             movie title  user2 rating  \
21        22                       Braveheart (1995)      4.139410   
99       100                            Fargo (1996)      4.596102   
63        64        Shawshank Redemption, The (1994)      4.452792   
49        50                        Star Wars (1977)      4.609644   
55        56                     Pulp Fiction (1994)      4.093374   
8          9                 Dead Man Walking (1995)      4.018546   
29        30                    Belle de jour (1967)      3.661514   
95        96       Terminator 2: Judgment Day (1991)      3.863171   
92        93         Welcome to the Dollhouse (1995)      3.946802   
90        91  Nightmare Before Christmas, The (1993)      3.769416   
69        70      Four Weddings and a Funeral (1994)      3.622047   
88        89                     Blade Runner (1982)      4.135199   
91        92                     True Romance (1993)      3.746381   
22        23                      Taxi Driver (1976)      4.212438   
82        83           Much Ado About Nothing (1993)      4.108987   
94        95                          Aladdin (1992)      3.617078   
11        12              Usual Suspects, The (1995)      4.446519   
1          2                        GoldenEye (1995)      3.090040   
68        69                     Forrest Gump (1994)      3.846700   
80        81             Hudsucker Proxy, The (1994)      3.509908   

    user17 rating  user35 rating   average   minimum     score  
21       4.124113       3.309282  4.131761  4.124113  3.826781  
99       4.819830       4.710713  4.707966  4.596102  3.632286  
63       4.512652       3.941884  4.482722  4.452792  3.616418  
49       4.626430       3.909395  4.618037  4.609644  3.391685  
55       4.294289       4.553403  4.193832  4.093374  3.368853  
8        4.183173       4.653366  4.100859  4.018546  3.310795  
29       3.921391       4.344089  3.791453  3.661514  3.275109  
95       3.562235       3.777007  3.712703  3.562235  3.240395  
92       3.913879       3.701667  3.930340  3.913879  3.226307  
90       3.933241       5.254208  3.851328  3.769416  3.143019  
69       3.879848       5.084023  3.750948  3.622047  3.062905  
88       4.088676       5.220215  4.111938  4.088676  3.000451  
91       3.919449       3.931506  3.832915  3.746381  2.997072  
22       4.373822       2.920528  4.293130  4.212438  2.987861  
82       4.022820       4.087805  4.065903  4.022820  2.930938  
94       3.703366       2.495453  3.660222  3.617078  2.923674  
11       4.682449       3.429083  4.564484  4.446519  2.917599  
1        3.036803       3.596877  3.063421  3.036803  2.881633  
68       3.580931       3.616974  3.713815  3.580931  2.849745  
80       3.820193       4.088311  3.665051  3.509908  2.840144

movie id                             movie title  user2 rating  \
99       100                            Fargo (1996)      4.596102   
21        22                       Braveheart (1995)      4.139410   
63        64        Shawshank Redemption, The (1994)      4.452792   
49        50                        Star Wars (1977)      4.609644   
55        56                     Pulp Fiction (1994)      4.093374   
8          9                 Dead Man Walking (1995)      4.018546   
11        12              Usual Suspects, The (1995)      4.446519   
92        93         Welcome to the Dollhouse (1995)      3.946802   
22        23                      Taxi Driver (1976)      4.212438   
88        89                     Blade Runner (1982)      4.135199   
82        83           Much Ado About Nothing (1993)      4.108987   
90        91  Nightmare Before Christmas, The (1993)      3.769416   
29        30                    Belle de jour (1967)      3.661514   
91        92                     True Romance (1993)      3.746381   
95        96       Terminator 2: Judgment Day (1991)      3.863171   
69        70      Four Weddings and a Funeral (1994)      3.622047   
97        98        Silence of the Lambs, The (1991)      4.318462   
94        95                          Aladdin (1992)      3.617078   
47        48                      Hoop Dreams (1994)      4.116555   
44        45              Eat Drink Man Woman (1994)      4.164948   

    user17 rating  user35 rating   average   minimum     score  
99       4.819830       4.710713  4.707966  4.596102  2.828365  
21       4.124113       3.309282  4.131761  4.124113  2.827471  
63       4.512652       3.941884  4.482722  4.452792  2.818970  
49       4.626430       3.909395  4.618037  4.609644  2.776823  
55       4.294289       4.553403  4.193832  4.093374  2.570014  
8        4.183173       4.653366  4.100859  4.018546  2.533803  
11       4.682449       3.429083  4.564484  4.446519  2.504906  
92       3.913879       3.701667  3.930340  3.913879  2.503043  
22       4.373822       2.920528  4.293130  4.212438  2.467849  
88       4.088676       5.220215  4.111938  4.088676  2.454341  
82       4.022820       4.087805  4.065903  4.022820  2.399162  
90       3.933241       5.254208  3.851328  3.769416  2.388672  
29       3.921391       4.344089  3.791453  3.661514  2.368326  
91       3.919449       3.931506  3.832915  3.746381  2.321882  
95       3.562235       3.777007  3.712703  3.562235  2.302738  
69       3.879848       5.084023  3.750948  3.622047  2.279219  
97       4.341874       4.435086  4.330168  4.318462  2.278085  
94       3.703366       2.495453  3.660222  3.617078  2.274588  
47       4.095128       4.152270  4.105841  4.095128  2.232198  
44       4.252097       4.319629  4.208522  4.164948  2.206581